### *IPCC SR15 scenario assessment*
<img style="float: right; height: 100px; margin-top: 10px;" src="../_static/IIASA_logo.png">
<img style="float: right; height: 80px;" src="../_static/IAMC_logo.jpg">

# Final energy demand and carbon intensity analysis

This notebook contains the scripts to generate **Figure 2.14** showing final energy demand development
and the carbon intensity across scenarios
for the IPCC's _"Special Report on Global Warming of 1.5°C"_.

The scenario data used in this analysis can be accessed and downloaded at [https://data.ene.iiasa.ac.at/iamc-1.5c-explorer](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer).

## Load `pyam` package and other dependencies

In [ ]:
import pandas as pd
import numpy as np
import warnings
import io
import itertools
import yaml
import math
import matplotlib.pyplot as plt
plt.style.use('style_sr15.mplstyle')
%matplotlib inline
import pyam

from utils import boxplot_by_cat

## Import scenario data, categorization and specifications files

The metadata file must be generated from the notebook `sr15_2.0_categories_indicators` included in this repository.  
If the snapshot file has been updated, make sure that you rerun the categorization notebook.

The last cell of this section loads and assigns a number of auxiliary lists as defined in the categorization notebook.

In [ ]:
sr1p5 = pyam.IamDataFrame(data='../data/iamc15_scenario_data_world_r1.xlsx')

In [ ]:
sr1p5.load_metadata('sr15_metadata_indicators.xlsx')

In [ ]:
with open("sr15_specs.yaml", 'r') as stream:
    specs = yaml.load(stream)

rc = pyam.run_control()
for item in specs.pop('run_control').items():
    rc.update({item[0]: item[1]})
cats = specs.pop('cats')
all_cats = specs.pop('all_cats')
subcats = specs.pop('subcats')
all_subcats = specs.pop('all_subcats')
marker= specs.pop('marker')

## Downselect scenario ensemble to categories of interest for this assessment

In [ ]:
years = [2020, 2030, 2050, 2070, 2100]

In [ ]:
df = sr1p5.filter(category=cats, year=years)

## Set specifications for filter and plotting

In [ ]:
save_name = 'output/fig2.14{}.{}'

In [ ]:
categories = cats.copy()
categories.remove('Above 2C')
column = 'category'

In [ ]:
filter_args = dict(df=sr1p5, category=categories, marker=None, join_meta=True)

In [ ]:
def plotting_args(name, filetype='png', hlines=[]):
    return {'categories': categories, 'column': 'category', 'years': years, 'add_marker': marker,
            'hlines': hlines, 'save': save_name.format(name, filetype)}

## Development of final energy demand by category

In [ ]:
fe = df.filter(variable='Final Energy').timeseries()
fe.index = fe.index.droplevel([2, 3, 4])

In [ ]:
name = 'final_energy'
boxplot_by_cat(pyam.filter_by_meta(fe, **filter_args),
               **plotting_args('a_{}'.format(name)),
               ylabel='Final Energy (EJ)')

## Share of electricity in final demand

In [ ]:
fe_ele = df.filter(variable='Final Energy|Electricity').timeseries()
fe_ele.index = fe_ele.index.droplevel([2, 3, 4])

In [ ]:
fe_ele_share = fe_ele / fe * 100

In [ ]:
name = 'ele_share'
boxplot_by_cat(pyam.filter_by_meta(fe_ele_share, **filter_args),
               **plotting_args('b_{}'.format(name)), legend=False,
               ylabel='Electricity share in Final Energy (%)')

## Plot the development of the carbon intensity of eletricity vs. the residual energy demand

In [ ]:
co2 = df.filter(variable='Emissions|CO2').timeseries()
co2.index = co2.index.droplevel([2, 3, 4])

In [ ]:
co2_ele = df.filter(variable='Emissions|CO2|Energy|Supply|Electricity').timeseries()
co2_ele.index = co2_ele.index.droplevel([2, 3, 4])

In [ ]:
carbon_intensity_ele = co2_ele / fe_ele

In [ ]:
name = 'carbon_intensity_electricity'
boxplot_by_cat(pyam.filter_by_meta(carbon_intensity_ele, **filter_args),
               **plotting_args('c_{}'.format(name), hlines=[0]), legend=False,
               ylabel='Carbon intensity of electricity (gCO2/MJ)')

In [ ]:
carbon_intensity_residual = (co2 - co2_ele) / (fe - fe_ele)

In [ ]:
name = 'carbon_intensity_residual'
boxplot_by_cat(pyam.filter_by_meta(carbon_intensity_residual, **filter_args),
               **plotting_args('d_{}'.format(name), hlines=[0]), legend=False,
               ylabel='Carbon intensity of residual fuel mix (gCO2/MJ)')